Pre Processing function

In [ ]:
remove_words = ["@handle", "RT", "http"]

def pre_process(sentence, max_length):
    sentence = sentence.split()
    target_remove = set()
    for token in sentence:
        for target in remove_words:
            if target in token:
                target_remove.add(token)
                
    for target in target_remove:
        sentence.remove(target)
    max_length = max(max_length, len(sentence))
    sentence = ' '.join(sentence)
    return sentence, max_length


Load Train data to train_dict. 
train_dict[id] = [[train_instace1], [train_instance2] ....]

In [ ]:
import csv
import collections

train_file_path = "data/train_tweets.txt"
train_dict = collections.defaultdict(list)
max_length = 0

with open(train_file_path, encoding='utf-8') as tsvfile:
    reader = csv.reader((x.replace('\0', '') for x in tsvfile), delimiter='\t')
    for i, row in enumerate(reader):
        id = int(row[0])
        instance, max_length = pre_process(row[1], max_length)
        if not instance == "":
            train_dict[id].append(instance)
    print("Total rows: %d" % i)
    
print("Total ids: %d" % len(train_dict))
print("Longest Sentence: %d" % (max_length))


In [ ]:
print(train_dict[1319])

Split Dev Train
!! There are Some ID have 0 train instance

In [ ]:
import random
import numpy as np

dev_split = 0.1
train_split = 1 - dev_split

dev_set_dict = {}
train_set_dict = {}

for id in train_dict:
    target_list = train_dict[id]
    length = len(target_list)
    random.shuffle(target_list)
    split = int(np.ceil(length*dev_split))
    dev_set_dict[id] = target_list[:split]
    train_set_dict[id] = target_list[split:length]
#     print(len(dev_set_dict[id]), len(train_set_dict[id]), length)

print(len(dev_set_dict), len(train_set_dict), len(train_dict))
    
    


Build ID idx and train/dev set save to file

In [ ]:
import pickle

def save_to_file(target_dict, file_path):
    file_lines = []
    for id in target_dict:
        for sentence in target_dict[id]:
            file_lines.append(str(id) + '\t' + sentence + '\n')
            
    random.shuffle(file_lines)
    print(len(file_lines))
#     with open(file_path, 'w+') as file:
#         for item in file_lines:
#             file.write(item)
    return

dev_set_path = 'data/v1/dev_set_v1.txt'
train_set_path = 'data/v1/train_set_v1.txt'
idx_file_path = 'data/v1/v1_idx.pickle'

save_to_file(dev_set_dict, dev_set_path)
save_to_file(train_set_dict, train_set_path)


# Build IDX and Save
idx = {}
for i, id in enumerate(train_dict):
    idx[id] = i
print(len(idx))

with open(idx_file_path, 'wb') as handle:
    pickle.dump(idx, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [29]:
from feature_extract_inference import FeatureExtract
test_model = FeatureExtract(checkpoints_path="/Users/hanxunhuang/Desktop/checkpoints/coconut_extract_model_v2_best.pth")

In [30]:
from numpy import dot
from numpy.linalg import norm

feature = test_model.get_features(["Shannon was right, Hamilton should step down #fb", "Listen live on as I debate @handle Friday Oct 2nd at 10:00 am #fb"])
feature1 = feature[0]
feature2 = feature[1]

cos_sim = dot(feature1, feature2)/(norm(feature1)*norm(feature2))
print(cos_sim)



0.42785403
